In [1]:
from hmmlearn import hmm

In [2]:

import nilmtk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import warnings

In [3]:
warnings.filterwarnings("ignore")


In [4]:
from nilmtk import *
import os
import nilmtk

In [35]:
st = pd.HDFStore(os.path.expanduser("/Users/nipunbatra/wiki-15min.h5"))

In [36]:
metadata_df = pd.read_csv("/Users/nipunbatra/w/dataport-metadata.csv",index_col=0)

In [37]:
feeds = {'use':'aggregate',
        'air1':'hvac',
         'clotheswasher1':'wm',
         'dishwasher1':'dw',
         'microwave1':'mw',
         'refrigerator1':'fridge',
         'oven1':'oven',
        'car1':'ec',
         'waterheater1':'wh',
         'drye1':'dr'
        }

In [38]:
cities = ['Austin','Houston','San Diego','Boulder']

In [39]:
months=12
start='1-1-2015'
start_month=1
end = '12-31-2015'
end_month=12

In [66]:
model = {}
for city in ['San Diego','Austin','Boulder']:
    print city
    print "*"*20
    city_data = metadata_df[metadata_df['city'] == city]
    city_homes = city_data.index.values.astype('int')
    city = city.replace(" ","")
    model[city] = {}
    for appliance in ["air1", "refrigerator1", "clotheswasher1", 
                      "dishwasher1", "microwave1", "car1","oven1","waterheater1","dryer1"]:
        print appliance, "training"
        if appliance in ["refrigerator1"]:
            activation_min = 0.05
        else:
            activation_min = 0.0
        o = []
        for home in city_homes[:50]:
            try:
                dft = st['/%d' %home][appliance][start:end]
                dft = dft.resample("15T", how="mean")
                appl_power = dft.dropna().values.reshape(-1,1)
                activation = (dft>10).sum()*1.0/len(dft)
                
            
                
                if activation>activation_min:
                    o.append(appl_power)
            except:
                pass
        if len(o)>1:
            o = np.array(o)
            mod = hmm.GaussianHMM(3, "full")
            mod.fit(o)
            model[city][feeds[appliance]] = mod
            print "Means for %s are" %appliance
            print mod.means_
        else:
            print "Not enough samples for %s" %appliance
        



San Diego
********************
air1 training
Means for air1 are
[[    0.        ]
 [  349.4476466 ]
 [ 2022.82781261]]
refrigerator1 training
Means for refrigerator1 are
[[  87.05974175]
 [   2.95077789]
 [ 221.86712369]]
clotheswasher1 training
Means for clotheswasher1 are
[[   2.1808973 ]
 [ 104.1043107 ]
 [  24.84099837]]
dishwasher1 training
Means for dishwasher1 are
[[   0.        ]
 [ 563.30253133]
 [  32.83688517]]
microwave1 training
Means for microwave1 are
[[   4.24549058]
 [ 145.44526921]
 [  39.70353194]]
car1 training
Not enough samples for car1
oven1 training
Means for oven1 are
[[    3.56979261]
 [ 2197.96146279]
 [  714.80359473]]
waterheater1 training
Not enough samples for waterheater1
dryer1 training
Not enough samples for dryer1
Austin
********************
air1 training
Means for air1 are
[[  3.00917924e-01]
 [  1.88513596e+03]
 [  4.19637327e+02]]
refrigerator1 training
Means for refrigerator1 are
[[ 110.60030677]
 [ 219.71444707]
 [  32.8040773 ]]
clotheswasher1 t

In [67]:
for city, model_city in model.iteritems():
    new_learnt_models = OrderedDict()
    for appliance, appliance_model in model_city.iteritems():
        startprob, means, covars, transmat = sort_learnt_parameters(
                        appliance_model.startprob_, appliance_model.means_,
                        appliance_model.covars_, appliance_model.transmat_)
        new_learnt_models[appliance] = hmm.GaussianHMM(
                    startprob.size, "full", startprob, transmat)
        new_learnt_models[appliance].means_ = means
        new_learnt_models[appliance].covars_ = covars
        from nilmtk.disaggregate.fhmm_exact import create_combined_hmm
        learnt_model_combined = create_combined_hmm(new_learnt_models)
        from nilmtk.disaggregate.fhmm_exact import FHMM
        f = FHMM()
        f.model = learnt_model_combined
        f.individual = new_learnt_models
        import pickle
        pickle.dump(f, open( "../data/model/fhmm_model_%s.p" %city, "wb" ))

        

In [70]:
import sys
#st_out = pd.HDFStore("../fhmm_out.h5")
done = -1
for home in [9932][:]:
    done = done +1
    #print "%0.2f" %(100*done/len(all_homes_all_appliance))
    sys.stdout.flush()
    try:
        mains_df = st['/%d' %home][['use']]['2015'].resample("15T")
        pred = f.disaggregate_chunk(mains_df).resample("1M", how="sum").mul(0.000017).mul(15)
        pred.index = pred.index.month
        #st_out['%d' %home] = pred
    except:
        pass

,dw,fridge,hvac,mw,oven,wm
1,70.995570,85.026690,99.140175,52.949475,96.884955,33.250470
2,76.909530,78.645060,99.055770,45.946665,101.755710,31.930845
3,53.438565,92.027205,102.535755,51.046410,47.652360,35.097945
4,38.342565,75.531510,114.373620,46.230480,36.121770,32.553045
5,45.995625,76.397490,147.467520,50.994900,57.339555,32.693295
6,49.665330,71.841150,176.370750,52.475175,144.855300,31.422120
7,74.013495,76.825125,311.352705,48.211065,294.082320,32.656575
8,39.657855,86.139765,122.418870,55.480860,161.660055,39.435495
9,54.430260,72.063765,190.190730,45.874755,201.196020,32.858790
10,53.163420,75.721230,143.987790,46.005570,58.914435,32.073390


In [73]:
st['/%d' %home]['2015'].resample("1M")

,use,air1,car1,clotheswasher_dryg1,dishwasher1,furnace1,oven1,range1
2015-01-31 00:00:00-06:00,576.074341,12.594064,49.919758,3.760058,18.610596,93.323586,8.154121,12.576097
2015-02-28 00:00:00-06:00,635.145264,12.484945,51.923164,5.707763,26.527506,88.582245,14.533607,20.762327
2015-03-31 00:00:00-05:00,507.586029,12.121441,23.933580,4.652216,19.035006,58.750748,7.112106,7.237900
2015-04-30 00:00:00-05:00,471.139526,11.727824,21.752731,4.818056,13.709491,25.912640,6.788773,5.581250
2015-05-31 00:00:00-05:00,545.507202,51.106632,24.956631,5.482191,15.457415,29.060036,1.833356,7.663934
2015-06-30 00:00:00-05:00,717.394226,253.810577,4.839421,3.935833,14.510672,59.977314,1.712917,6.007222
2015-07-31 00:00:00-05:00,1106.334473,519.911194,11.034879,5.068907,19.040995,102.850243,1.135865,10.264539
2015-08-31 00:00:00-05:00,665.150940,262.374451,3.679861,2.057415,5.826568,59.072670,0.011559,3.201299
2015-09-30 00:00:00-05:00,812.180115,316.764496,4.091366,6.286366,9.844746,66.997643,2.285741,6.180764
2015-10-31 00:00:00-05:00,539.902344,51.870899,4.186245,2.772043,8.785350,25.599798,7.701389,12.401635
